In [1]:
from langchain_community.tools import YouTubeSearchTool
from youtube_transcript_api import YouTubeTranscriptApi
import ast
import re
from langchain_together import ChatTogether
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.output_parsers.json import JsonOutputParser
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph import  END
from gtts import gTTS

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
llm = ChatTogether(
    model = "meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature = 0
)

In [4]:
text = "Report mAIstro for multi-agent research"

In [5]:
class State(TypedDict):
    video_name: str
    url: str
    video_id: str
    transcript: str
    key_points: str
    summary: str
    audio_file: str

In [6]:
graph = StateGraph(State)

In [7]:
async def get_video_url(state: State):
    """
    Searches YouTube for a given query and returns a video URL.

    Args:
        query (str): The search query.

    Returns:
        str: A YouTube video URL, or None if no results are found.
    """
    refined_query = state['video_name']+", 1"
    tool = YouTubeSearchTool()
    result = tool.run(refined_query)

    try:
        result_list = ast.literal_eval(result)  # Convert string to list
        return {"url": result_list[0]} if result_list else {"url": None}  
    except (SyntaxError, ValueError):
        print("Error parsing YouTube search results.")
        return {"url": None}

In [8]:
async def get_video_id_from_url(state: State) -> str:
    """
    Extracts the video ID from a YouTube URL.
    
    Args:
        youtube_url (str): The full YouTube video URL.

    Returns:
        str: The extracted video ID, or None if not found.
    """
    match = re.search(r"(?:v=|\/embed\/|\/v\/|\/vi\/|youtu\.be\/|\/e\/|watch\?v=|&v=)([a-zA-Z0-9_-]{11})", state['url'])
    return {"video_id": match.group(1)} if match else {"video_id": None}

In [9]:

async def transcribe_youtube_video(state: State):
    """
    Fetches and transcribes a YouTube video using its video ID.

    Args:
        video_id (str): The YouTube video ID.

    Returns:
        str: The transcript text, or an error message if unavailable.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(state['video_id'])
        transcript_text = " ".join([entry["text"] for entry in transcript])
        return {"transcript": transcript_text}
    except Exception as e:
        print(f"Error fetching transcript: {str(e)}")
        return {"transcript": None}

In [10]:
async def get_video_summary(state: State):
    """
    Generates a summary of a YouTube video transcript.

    This function takes a video transcript stored in the given state 
    and processes it using an AI-powered summarization pipeline. 
    The summary captures the core points and main message of the video.

    Args:
        state (State): A state object containing the 'transcript' key 
                      with the video transcript as its value.

    Returns:
        dict: A dictionary containing the summary of the video 
              with the key 'summary'.
    """
    summarizer_prompt = """
    You are a video summarizer that summarizes YouTube video transcripts. 
    Given this transcription: {transcription}
    You are to create a summary that includes includes the crux of the video and contains the core points of the video.
    """
    summarizer_prompt_template = PromptTemplate(
        input_variables = ['transcripiton'],
        template = summarizer_prompt
    )
    summarizer_chain = summarizer_prompt_template | llm | StrOutputParser()
    summary = await summarizer_chain.ainvoke({"transcription": state['transcript']})
    
    return {"summary": summary}

In [11]:
async def generate_audio_summary(state: State):
    """
    Converts the video summary into an audio file.

    Args:
        state (State): Contains the 'summary' key with the summary text.

    Returns:
        dict: Dictionary containing the filename of the generated audio file.
    """
    summary_text = state["summary"]
    
    if not summary_text:
        return {"audio_file": None}

    try:
        audio_filename = "summary.mp3"
        tts = gTTS(text=summary_text, lang="en")
        tts.save(audio_filename)
        return {"audio_file": audio_filename}
    except Exception as e:
        print(f"Error generating audio: {e}")
        return {"audio_file": None}


In [12]:
async def get_video_keypoints(state: State):
    """
    Generates a summary of a YouTube video transcript.

    This function takes a video transcript stored in the given state 
    and processes it using an AI-powered summarization pipeline. 
    The summary captures the core points and main message of the video.

    Args:
        state (State): A state object containing the 'transcript' key 
                      with the video transcript as its value.

    Returns:
        dict: A dictionary containing the summary of the video 
              with the key 'summary'.
    """
    key_points_prompt = """
    You are a transcript Note-Taker that give the key-points of a YouTube video transcripts. 
    Given this transcription: {transcription}
    You are to give 5 crucial keypoints that hit the crux of the video and contains the core points of the video.
    """
    key_points_prompt_template = PromptTemplate(
        input_variables = ['transcripiton'],
        template = key_points_prompt
    )
    
    key_points_chain = key_points_prompt_template | llm | StrOutputParser()
    key_points = await key_points_chain.ainvoke({"transcription": state['transcript']})
    
    return {"key_points": key_points}

In [13]:
async def sentiment_analysis(state: State):
    pass

In [14]:
def transcript_condition(state):
    """Decides the next step based on transcript availability."""
    transcript = state.get("transcript")
    if transcript is None or transcript.startswith("Error"):  
        return END  
    return "get_video_summary"

In [15]:
graph.add_node("get_video_url", get_video_url)
graph.add_node("get_video_id_from_url", get_video_id_from_url)
graph.add_node("transcribe_youtube_video", transcribe_youtube_video)
graph.add_node("get_video_summary", get_video_summary)
graph.add_node("generate_audio_summary", generate_audio_summary)
graph.add_node("get_video_keypoints", get_video_keypoints)

In [16]:
graph.add_edge("get_video_url", "get_video_id_from_url")
graph.add_edge("get_video_id_from_url", "transcribe_youtube_video")
graph.add_conditional_edges("transcribe_youtube_video", transcript_condition)
graph.add_edge("transcribe_youtube_video", "get_video_summary")
graph.add_edge("transcribe_youtube_video", "get_video_keypoints")
graph.add_edge("get_video_summary", "generate_audio_summary")

graph.add_edge("generate_audio_summary", END)
graph.add_edge("get_video_keypoints", END)

# Define entry and output nodes
graph.set_entry_point("get_video_url")

# Compile the LangGraph workflow
transcriber_workflow = graph.compile()

In [17]:

# Function to run the workflow
async def run_transcriber(video_name: str):
    initial_state = {"video_name": video_name, "url": "", "video_id": "", "transcript": "", "key_points": "", "summary": "", "audio_file": ""}
    final_state = await transcriber_workflow.ainvoke(initial_state)
    return final_state

In [18]:
query = "Golang FAILED Me! Here's What Worked Instead"
result = await run_transcriber(query)

In [19]:
result

{'video_name': "Golang FAILED Me! Here's What Worked Instead",
 'url': 'https://www.youtube.com/watch?v=t8HDXwgn0I8&pp=ygUsR29sYW5nIEZBSUxFRCBNZSEgSGVyZSdzIFdoYXQgV29ya2VkIEluc3RlYWQ%3D',
 'video_id': 't8HDXwgn0I8',
 'transcript': "what's going on ladies and gentlemen this video is going to be a banger because go sucks of course not so um for the people that don't know this is basically a video that has been requested uh for a for multiple times now is why do I need to basically move to another language um instead of using go for my client right and what is the language I basically picked is it is it C++ is it Odin is it Z I basically used all of them and uh you want to see if you want to if you want to know what kind of a uh what what language it will be definitely you need to basically check uh need to stick around right don't don't don't leave okay uh for the people but before we continue for the people that like the videos I am providing consider subscribing to my channel you would

In [20]:
print(result['key_points'])

Here are the 5 crucial keypoints that summarize the video:

1. **The creator is moving away from using Go for their client-side application**: The creator of the Market Monkey project is looking to replace Go with another language for their client-side application due to performance issues with WebAssembly (WASM) compilation.

2. **Go is not suitable for low-level, high-performance applications**: The creator notes that while Go is great for backend systems and networking, it's not ideal for low-level, high-performance applications that require direct access to hardware resources, which is a limitation they're experiencing with their project.

3. **The creator is exploring alternative languages, including C++, Zig, and Odin**: The creator is considering alternative languages to replace Go, including C++, Zig, and Odin, each with their own strengths and weaknesses, and is currently leaning towards using Odin due to its familiarity and low-level capabilities.

4. **Odin is the chosen lan

In [21]:
print(result['summary'])

The video discusses the creator's experience with the Go programming language and its limitations, particularly when it comes to building a native desktop application that can be compiled to WebAssembly (WASM). The creator is building a project called Market Monkey, a trading terminal for cryptocurrencies, and initially used Go for both the backend and client-side. However, they encountered performance issues when compiling Go to WASM, which led them to explore alternative languages.

The creator considered using C++, Rust, Zig, and Odin, but ultimately decided against Rust due to personal preference. They experimented with Zig, which they found to be a great language with an excellent build system, but ultimately decided it wasn't the right fit for them. They also explored Odin, which they found to be a great language that writes like Go and has a similar style to Pascal.

The creator decided to use Odin for their project, despite its limited tooling and build system, because it allow